In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install llama-index llama-index-embeddings-huggingface chromadb llama-index-vector-stores-chroma llama-index-postprocessor-cohere-rerank pip install llama-index-llms-huggingface --quiet
!pip install transformers accelerate bitsandbytes --quiet

!pip install gtts
!pip install librosa scipy gradio Cython --quiet
!apt-get update && apt-get install -y libsndfile1 ffmpeg
!pip install nemo_toolkit['all'] --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 525.5/525.5 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.4/15.4 MB 31.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 37.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.5/171.5 kB 20.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 92.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.9/91.9 kB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 53.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 68.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.1/60.1 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.1/106.1 kB 14.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [3]:
import torch
from transformers import BitsAndBytesConfig
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.llms.huggingface import HuggingFaceLLM
from llama_index.core.query_engine.retriever_query_engine import RetrieverQueryEngine

# load embedding model, for encoding database and query text
embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-large-en-v1.5")

# quantize to save memory
nf4_config = BitsAndBytesConfig(
   load_in_4bit=True,
   bnb_4bit_quant_type="nf4",
   bnb_4bit_use_double_quant=True,
   bnb_4bit_compute_dtype=torch.bfloat16
)

model_id = "mistralai/Mistral-7B-Instruct-v0.2"
llm = HuggingFaceLLM(
    model_name=model_id,
    tokenizer_name=model_id,
    context_window=32768,
    max_new_tokens=128,
    model_kwargs={"quantization_config": nf4_config},
)

# initializing global setting for the whole project
from llama_index.core import Settings
from llama_index.core.node_parser import SentenceSplitter

Settings.llm = llm
Settings.embed_model = embed_model
Settings.chunk_size = 128
Settings.chunk_overlap = 32
Settings.transformations = [SentenceSplitter(chunk_size=128, chunk_overlap=32)]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/94.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.46k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

In [4]:
# load data into vector store
from pathlib import Path
from llama_index.readers.file import CSVReader
from llama_index.core import SimpleDirectoryReader

csv_loader = CSVReader()
csvs = csv_loader.load_data(file=Path("/content/drive/MyDrive/Colab Notebooks/data/csv/HKU_canteen_menu.csv"),extra_info={})
pdfs = SimpleDirectoryReader("/content/drive/MyDrive/Colab Notebooks/data/pdf").load_data()
documents = pdfs + csvs
metadata_dicts = [
    {
        "file_name": "HKU_canteen_admin_info.pdf",
        "information_contained": "Address, opening hours, closing work days",
    },
    {
        "file_name": "HKU_Canteen_menu,csv",
        "information_contained": "Cateen name, Item/dish types, Item/dish names, Item/dish prices, Whether item/dish is vegan friendly or not, and optional description about the possible choice of swapping ingredients",
    },
]
for idx, doc in enumerate(documents):
    doc.metadata = metadata_dicts[idx]

In [5]:
import chromadb
from llama_index.vector_stores.chroma import ChromaVectorStore
from llama_index.core import StorageContext, VectorStoreIndex
from llama_index.core.retrievers import VectorIndexRetriever
from llama_index.core.postprocessor import SentenceTransformerRerank, SimilarityPostprocessor
from llama_index.core import get_response_synthesizer

db = chromadb.EphemeralClient()
try:
    db.delete_collection("FYP_Canteen_VA")
except ValueError:
    # print("collection does not exist")
    pass
finally:
    chroma_collection = db.create_collection("FYP_Canteen_VA")
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
storage_context = StorageContext.from_defaults(vector_store=vector_store)

# create the vector store index that stores embedded documents
index = VectorStoreIndex.from_documents(documents, storage_context=storage_context, show_progress=True)
print(f"There is total {chroma_collection.count()} embedding entries with dimension {len(embed_model.get_text_embedding('1'))}")

retriever = VectorIndexRetriever(
    index=index,
    similarity_top_k=5,
)

response_synthesizer = get_response_synthesizer(
    response_mode="compact"
)

rerank = SentenceTransformerRerank(
    model='BAAI/bge-reranker-v2-m3', top_n=2
)

filter  = SimilarityPostprocessor(
    similarity_cutoff=0.0035
)

query_engine = RetrieverQueryEngine(
    retriever=retriever,
    response_synthesizer=response_synthesizer,
    node_postprocessors=[rerank, filter],
    # node_postprocessors=[rerank],
)

Parsing nodes:   0%|          | 0/2 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/99 [00:00<?, ?it/s]

There is total 99 embedding entries with dimension 1024


config.json:   0%|          | 0.00/795 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.27G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.17k [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/964 [00:00<?, ?B/s]

In [6]:
from IPython.display import Markdown, display
def display_prompt_dict(prompts_dict):
    for k, p in prompts_dict.items():
        text_md = f"**Prompt Key**: {k}<br>" f"**Text:** <br>"
        display(Markdown(text_md))
        print(p.get_template())
        display(Markdown("<br><br>"))

In [7]:
# update the prompt template
import json
from llama_index.core import PromptTemplate
from llama_index.core.schema import TextNode

zero_shot_prompt_template = """\
You are an HKU Canteen Ambassador, and your role is to provide friendly and helpful information about the canteens and their menus. \
Given the contexts and not prior knowledge, answer the question. You should answer the question in complete sentences \
and you should not answer in point form. The answer should have just enough content to answer the question.

Context: {context_str}
Question: {query_str}
Answer: \
"""

with open("/content/drive/MyDrive/Colab Notebooks/FYP_Eval/Dataset/few_shot_examples.json", "r")  as f:
    few_shot_examples = json.load(f)
    example_texts = []
    for example in few_shot_examples:
        input = example["input"]
        retrieval_context = " ".join(example["retrieval_context"]).replace("\n", "")
        expected_output = example["expected_output"]
        example_text  = f"""\
        Context: {retrieval_context}
        Question: {input}
        Answer: {expected_output}"""
        example_texts.append(example_text)

# reading few-shot examples
def return_few_shot_exampels(**kwargs):
    return "\n\n".join(example_texts)

# print("\n\n".join(example_texts))

few_shot_prompt_template = """\
You are an HKU Canteen Ambassador, and your role is to provide friendly and helpful information about the canteens and their menus. \
Given the contexts and not prior knowledge, answer the question. You should answer the question in complete sentences \
and you should not answer in point form. The answer should have just enough content to answer the question. Some examples are given below.

{few_shot_examples}

Context: {context_str}
Question: {query_str}
Answer: \
"""

query_engine.update_prompts(
    # {"response_synthesizer:text_qa_template": PromptTemplate(zero_shot_prompt_template)}
    {"response_synthesizer:text_qa_template": PromptTemplate(few_shot_prompt_template, function_mappings={"few_shot_examples":return_few_shot_exampels})}
)
prompts = query_engine.get_prompts()
display_prompt_dict(prompts)

**Prompt Key**: response_synthesizer:text_qa_template<br>**Text:** <br>

You are an HKU Canteen Ambassador, and your role is to provide friendly and helpful information about the canteens and their menus. Given the contexts and not prior knowledge, answer the question. You should answer the question in complete sentences and you should not answer in point form. The answer should have just enough content to answer the question. Some examples are given below.

{few_shot_examples}

Context: {context_str}
Question: {query_str}
Answer: 


<br><br>

**Prompt Key**: response_synthesizer:refine_template<br>**Text:** <br>

The original query is as follows: {query_str}
We have provided an existing answer: {existing_answer}
We have the opportunity to refine the existing answer (only if needed) with some more context below.
------------
{context_msg}
------------
Given the new context, refine the original answer to better answer the query. If the context isn't useful, return the original answer.
Refined Answer: 


<br><br>

In [8]:
user_input = "Where can I buy iced lemon tea?"
response_object = query_engine.query(user_input)

if response_object is not None:
    actual_output = response_object.response
    retrieval_context = [node.get_content() for node in response_object.source_nodes]
print(actual_output.strip().split("\n\n")[0])
print((" ".join(retrieval_context)).replace("\n", ""))

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


iced lemon tea is available at Kiosk by The Sandwich Club.
$13.0, is not vegan friendly, Kiosk by The Sandwich Club, Beverages, Iced Lemon Tea, $13.0, is not vegan friendly, Kiosk by The Sandwich Club, Beverages, Iced Lemon Water, $13.0, is not vegan friendly, Kiosk by The Sandwich Club, Beverages, Iced Hazelnut Latte, $25.0, is not vegan friendly, Kiosk by The Sandwich Club, Beverages, Iced Milk Tea, $13.0, is not vegan friendly, Kiosk by The Sandwich Club, Beverages, Iced Lemon Tea, $13.0,


In [9]:
# this section is to generate evaluation dataset based on a csv of query & expected answer pairs
# The dataset has the format of json with fields: input, actual_output, retrieval_context, expected_output
# Personally, I included node scores for monitoring purpose for the reject threshold
DEFAULT_ANSWER = "I'm afraid I don't have enough information to provide a satisfactory answer to your question. \
As an AI assistant, I'm committed to giving you accurate and factual responses based on the contexts I received. \
I don't wish to risk providing you with incorrect or misleading information. \
Please feel free to rephrase your question or ask about something else, and I'll do my best to help."

# # loading questions for evaluation
# with open("/content/drive/MyDrive/Colab Notebooks/FYP_Eval/Questions/FYP_questions.json", "r") as eval_questions:
#     question_reader = json.load(eval_questions)
#     eval_dataset = []

#     # headers: ID, query type, query content, question, expected answer
#     for idx, q in enumerate(question_reader):
#             user_query, output = q["input"], q["expected_output"]
#             response_object = query_engine.query(user_query)
#             nodes = response_object.source_nodes
#             if len(nodes) == 0: # refuse to answer
#                 temp = {"id": idx+1, "input": user_query, "actual_output": DEFAULT_ANSWER, "retrieval_context":  [], "expected_output": output, "scores": []}
#             else:
#                 temp = {"id": idx+1, "input": user_query, "actual_output": response_object.response.strip().split("\n\n")[0], "retrieval_context":  [(node.get_content()).replace("\n", "") for node in nodes], "expected_output": output, "scores": ["{:.4f}".format(float(node.score)) for node in nodes]}
#             eval_dataset.append(temp)

# # with open('/content/drive/MyDrive/Colab Notebooks/FYP_Eval/Dataset/threshold_eval_dataset.json', 'w') as json_file: # 5-6 mins inference time
# # with open('/content/drive/MyDrive/Colab Notebooks/FYP_Eval/Dataset/zero_shot_eval_dataset.json', 'w') as json_file: # 5-6 mins inference time
# with open('/content/drive/MyDrive/Colab Notebooks/FYP_Eval/Dataset/few_shot_eval_dataset.json', 'w') as json_file: # 12-13 mins inference time
# # with open('/content/drive/MyDrive/Colab Notebooks/FYP_Eval/Dataset/one_shot_eval_dataset.json', 'w') as json_file: #
#     # Write the eval dataset in the form of JSON
#     json.dump(eval_dataset, json_file)

In [10]:
# import ASR model
import nemo.collections.asr as nemo_asr
asr_model = nemo_asr.models.EncDecCTCModel.from_pretrained(model_name="nvidia/parakeet-ctc-0.6b")

# import TTS module from Google
from gtts import gTTS

[NeMo W 2024-04-18 05:40:27 nemo_logging:349] /usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
      self.pid = os.fork()
    


parakeet-ctc-0.6b.nemo:   0%|          | 0.00/2.44G [00:00<?, ?B/s]

[NeMo I 2024-04-18 05:41:35 mixins:172] Tokenizer SentencePieceTokenizer initialized with 1024 tokens


[NeMo W 2024-04-18 05:41:35 modelPT:165] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    manifest_filepath: /disk1/NVIDIA/datasets/LibriSpeech_NeMo/librivox-train-all.json
    sample_rate: 16000
    batch_size: 16
    shuffle: true
    num_workers: 8
    pin_memory: true
    use_start_end_token: false
    trim_silence: false
    max_duration: 16.7
    min_duration: 0.1
    is_tarred: false
    tarred_audio_filepaths: null
    shuffle_n: 2048
    bucketing_strategy: fully_randomized
    bucketing_batch_size: null
    
[NeMo W 2024-04-18 05:41:35 modelPT:172] If you intend to do validation, please call the ModelPT.setup_validation_data() or ModelPT.setup_multiple_validation_data() method and provide a valid configuration file to setup the validation data loader(s). 
    Validation config : 
    manifest_filepath: /disk1/NVIDIA/datasets/LibriSpee

[NeMo I 2024-04-18 05:41:35 features:289] PADDING: 0
[NeMo I 2024-04-18 05:41:58 save_restore_connector:249] Model EncDecCTCModelBPE was successfully restored from /root/.cache/huggingface/hub/models--nvidia--parakeet-ctc-0.6b/snapshots/097ffc5b027beabc73acb627def2d1d278e774e9/parakeet-ctc-0.6b.nemo.


In [11]:
import gradio as gr
import numpy as np
from transformers import pipeline
import librosa
from scipy.io.wavfile import write

def user(user_query, chat_history):
    return "", [[user_query, None]]

def respond(chat_history):
    user_query = chat_history[-1][0]
    response_object = query_engine.query(user_query)
    if len(response_object.source_nodes) == 0:
      bot_response = DEFAULT_ANSWER
    else:
      bot_response = response_object.response.strip().split("\n\n")[0]
    chat_history.append([None, bot_response])
    return chat_history

def transcribe(audio):
    try:
        sr, y = audio
        audio_name = "resampled_audio.wav"
        resampled_audio = librosa.resample(y=y.astype("float"), orig_sr=sr, target_sr=16000)
        write(audio_name, 16000, resampled_audio)
        text = asr_model.transcribe([f"./{audio_name}"])
    except:
        text = [""]
    return text[0]

def text_to_speech(chatbot):
    try:
        path = "/content/drive/MyDrive/Colab Notebooks/FYP_TTS/temp.wav"
        tts = gTTS(chatbot[-1][1], lang="en", tld="com.hk")
        tts.save(path)
    except:
        # default tts if failed
        path = "/content/drive/MyDrive/Colab Notebooks/FYP_TTS/default.wav"
        tts = gTTS("Sorry, there is something wrong with TTS. Maybe you can try again later.", lang="en", tld="com.hk")
        tts.save(path)
    return path

with gr.Blocks() as demo:
    with gr.Column():
        gr.Markdown(
        """
        # HKU Canteen Voice Assistant
        """)
        chatbot = gr.Chatbot(container=False) # change to QA not chatbot

        with gr.Row(): # text input
            text_input = gr.Textbox(placeholder="Ask me anything...", container=False, scale=1)
            submit_btn = gr.Button("Submit", scale=0)

        with gr.Row():  # audio input
            recording = gr.Microphone(label="Record your question", show_label=True, show_download_button=False)

        with gr.Row(): # audio output
            tts = gr.Audio(label="Text to speech response", show_label=True)

        with gr.Row(): # button toolbar
            clear = gr.ClearButton([text_input, chatbot])

        text_input.submit(user, [text_input, chatbot], [text_input, chatbot], queue=False).then(respond, [chatbot], [chatbot]).then(text_to_speech, [chatbot], [tts])
        submit_btn.click(user, [text_input, chatbot], [text_input, chatbot], queue=False).then(respond, [chatbot], [chatbot]).then(text_to_speech, [chatbot], [tts])
        recording.stop_recording(transcribe, [recording], [text_input])

if __name__ == "__main__":
    demo.launch(debug=True, share=True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://6ba9733ba9d2bba36f.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


...

Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://6ba9733ba9d2bba36f.gradio.live
